In [1]:
!pip install sentence-transformers pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, evaluation
from torch.utils.data import DataLoader
import pandas as pd
import os
import math

# --- 1. CONFIGURAZIONE ---
model_name = 'all-MiniLM-L6-v2'  # Il modello base
train_batch_size = 16            # Numero di coppie per batch (più alto è meglio per questa Loss, ma occhio alla memoria)
num_epochs = 4                   # Numero di passaggi sui dati (non esagerare per evitare overfitting su dataset piccoli)
model_save_path = 'FineTunedModel/fine_tuned_compliance_model' # Cartella dove salvare il modello
csv_file = 'TrainAndTestData/training.csv' # Il file generato nello step precedente

# Verifica esistenza file
if not os.path.exists(csv_file):
    raise FileNotFoundError(f"Il file {csv_file} non esiste! Esegui prima il codice di generazione dati.")

# --- 2. CARICAMENTO DATI ---
print("Caricamento dataset...")
df = pd.read_csv(csv_file)

# Convertiamo il DataFrame in una lista di oggetti InputExample
train_examples = []
for i, row in df.iterrows():
    # InputExample accetta una lista di testi. Qui ne passiamo due: [Anchor, Positive]
    train_examples.append(InputExample(texts=[str(row['anchor']), str(row['positive'])]))

print(f"Dataset caricato: {len(train_examples)} coppie di training.")

# Creazione del DataLoader
# Shuffle è importante!
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

# --- 3. INIZIALIZZAZIONE MODELLO ---
print(f"Scaricamento modello base: {model_name}...")
model = SentenceTransformer(model_name)

# --- 4. DEFINIZIONE DELLA LOSS FUNCTION ---
# MultipleNegativesRankingLoss è ottima quando hai solo coppie positive.
# Usa gli altri campioni nel batch come negativi impliciti.
train_loss = losses.MultipleNegativesRankingLoss(model=model)

# --- 5. CALCOLO PASSI DI WARMUP ---
# Il warmup aiuta a stabilizzare il training all'inizio
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% del training

# --- 6. TRAINING ---
print("Inizio del fine-tuning...")
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    show_progress_bar=True,
    output_path=model_save_path # Salva automaticamente qui alla fine
)

print(f"\n✅ Training completato! Il modello è stato salvato in: '{model_save_path}'")

/home/fabio.pinelli/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2025-11-25 15:30:51.664895: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 15:30:51.830128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764084651.903566 2925350 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764084651.927900 2925350 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alre

Caricamento dataset...
Dataset caricato: 574 coppie di training.
Scaricamento modello base: all-MiniLM-L6-v2...
Inizio del fine-tuning...


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/36 [00:00<?, ?it/s]

/home/fabio.pinelli/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Iteration:   0%|          | 0/36 [00:00<?, ?it/s]

Iteration:   0%|          | 0/36 [00:00<?, ?it/s]

Iteration:   0%|          | 0/36 [00:00<?, ?it/s]


✅ Training completato! Il modello è stato salvato in: 'FineTunedModel/fine_tuned_compliance_model'
